In [28]:
# Markdown

#This notebook will be used to make my version of the Coursera/IBM Capstone project for Applied Data Science.
#It will be about the boroughs and neighborhoods of Toronto, Canada
#Steps are as follows:
 #   1. Scrape the neighborhoods from wikipedia and make them into a pandas dataframe
 #   2. Using the Geocoder package, add latitude & longitude to each neighborhood
 #   3. Using the Foursquare API, cluster the different neighborhoods of Toronto and use Folium maps to visualize    

In [29]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [53]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

source = requests.get(url).text
wiki_page = BeautifulSoup(source, "lxml") #this is the whole page
table = wiki_page.find("table", class_ = "wikitable sortable") #this is only the table

# print(table.prettify())


In [54]:
#So I actually got the table in my variable called "table", but it's not a dataframe yet.
#Having loaded the pandas library already, I'll convert it into a readable dataframe first.

row= []
for tr in table.find_all('tr'): #finding "tr" as that indicates a new row
    data = tr.find_all('td') #Finding "td" as that indicates a new cell
    row.append([i.text for i in data]) #and append the cells to a new row in the list I'm creating
       
tbl = pd.DataFrame( data  = row ) #convert the list into a dataframe
tbl.head()

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [55]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood --> OK, let's do that
tbl.columns=["PostalCode", "Borough", "Neighborhood"]
tbl.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [56]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
tbl = tbl.drop([0]) #remove the 1st row with "none" all along
tbl = tbl[tbl.Borough != "Not assigned"]
tbl.head()


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n


In [57]:
#Before I move on, I will have to remove the "\n" at the end to each line
tbl["Neighborhood"] = tbl["Neighborhood"].str.strip()
tbl.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [58]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
# These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

tbl = tbl.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(', '.join).reset_index()
tbl.head()

#I got this solution from 
# https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby/45925961

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
# So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

tbl.loc[tbl.Neighborhood == "Not assigned", "Neighborhood"] = tbl.Borough

#This turns out to actually only change 1 row, so I'm not printing the head() this time, but the specific row:

#print("the new row looks like")
tbl[tbl["PostalCode"]=="M7A"]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [61]:
# Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

tbl.shape

(103, 3)